In [5]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [6]:
# read data

df = pd.read_csv('all_task_environments_new.csv', header=0, skiprows=[1])

### Define functions for prediction

In [7]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [86]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,30):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [87]:
def get_avg_editdist(data):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
    dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]
        
    for row in range(0,len(df)):
        objects = list(df.at[row,'objects'].split(','))
        strong_k = list(df.at[row,'strong_k'].split(','))
        mid_k = list(df.at[row,'mid_k'].split(','))
        food_k = list(df.at[row,'food_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
        start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
        sequence = str(df.at[row,'sequence'])
        
        for k2 in np.arange(1.1,2.0,0.1):
            k_food = round(k2,2)
            k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
            for k in np.arange(0,0.9,0.1):
                k_strong = round(k,2)
                k_mid = round(k + 0.1,2)
                k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                for c in np.arange(1.0,2.0,0.1):
                    c = round(c, 1)
                    c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
                
                    for dim in dimensions:                
                        # get average edit distance
                        median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence)
                        median = median / len(sequence)
                        params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                        results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
                    
    return results

### Calculate edit distances, create df

In [88]:
#%%timeit -n1 -r1
# 15 min for n = 30
%time results_new = get_avg_editdist(df)

CPU times: user 16min 16s, sys: 188 ms, total: 16min 16s
Wall time: 16min 16s


In [89]:
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.8; k: 0.8,0.9,1.9; yz","c: 1.8; k: 0.8,0.9,1.9; xyz","c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence
0,0.400000,0.800000,0.600000,0.300000,0.200000,0.400000,0.000000,0.200000,0.800000,0.600000,...,0.400000,0.200000,0.200000,0.600000,0.400000,0.000000,0.400000,0.400000,0.200000,tnpsc
1,0.400000,0.600000,0.600000,0.200000,0.400000,0.600000,0.200000,0.200000,0.600000,0.700000,...,0.600000,0.400000,0.200000,0.600000,0.400000,0.400000,0.600000,0.600000,0.400000,tnpcs
2,0.400000,0.600000,0.600000,0.000000,0.200000,0.600000,0.000000,0.200000,0.600000,0.800000,...,0.400000,0.200000,0.200000,0.400000,0.400000,0.200000,0.400000,0.400000,0.200000,tnpsc
3,0.600000,0.400000,0.400000,0.200000,0.400000,0.400000,0.200000,0.400000,0.400000,0.400000,...,0.200000,0.000000,0.400000,0.200000,0.300000,0.000000,0.000000,0.200000,0.000000,tnspc
4,0.333333,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.000000,0.666667,0.333333,...,0.333333,0.333333,0.000000,0.666667,0.166667,0.333333,0.000000,0.500000,0.333333,pms
5,0.600000,0.400000,0.800000,0.400000,0.400000,0.400000,0.400000,0.600000,0.200000,0.800000,...,0.800000,0.800000,0.600000,0.400000,0.800000,0.800000,0.400000,0.800000,0.800000,rhcse
6,0.875000,0.625000,1.000000,1.000000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.500000,0.500000,0.500000,0.500000,1.000000,0.500000,0.500000,0.500000,0.500000,pbhs
7,0.500000,1.000000,0.750000,1.000000,0.500000,1.000000,1.000000,1.000000,1.000000,0.750000,...,1.000000,1.000000,1.000000,1.000000,0.750000,1.000000,1.000000,1.000000,1.000000,hscp
8,0.333333,0.000000,0.333333,0.000000,0.166667,0.666667,0.000000,0.333333,0.666667,0.333333,...,0.333333,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,pbs
9,0.000000,0.666667,0.333333,0.333333,0.166667,0.333333,0.333333,0.333333,0.500000,0.000000,...,0.333333,0.333333,0.000000,0.666667,0.000000,0.333333,0.000000,0.333333,0.333333,pgs


### Get lowest error, compare edit distances

In [93]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [94]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

(0.36980820105820117, Index(['c: 1.5; k: 0.4,0.5,1.5; xy'], dtype='object'))

In [113]:
np.nanmean(list_mean), np.nanmedian(list_mean)

(0.5030064772822709, 0.5045882936507937)

In [125]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.4293229881115683, stdev: 0.020665739808410827, median: 0.42929894179894185
Average xyz: 0.4304333031223463, stdev: 0.01695306853215557, median: 0.42860449735449735


In [127]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.49664960154157684, min: 0.4076554232804233
Average y: 0.5428056821150957, min: 0.4522156084656085
Average z: 0.5902687757201646, min: 0.5093915343915344
Average xz: 0.497021380397152
Average yz: 0.5345436099679927


### Statistical analysis

In [128]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 4222.316, p =  0.000E+00


In [133]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 153435.500, p = 0.13253


### Plots for dataframe

In [171]:
len([x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)])

5670

In [173]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence']
dim = [x.strip() for x in results_new.columns[:-1].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [174]:
#%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + 0.603
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

### Check for sequences worse than baseline

In [175]:
# get sequences from best model where performance worse than baseline
results_new['c: 1.5; k: 0.4,0.5,1.5; xy'].loc[results_new['c: 1.5; k: 0.4,0.5,1.5; xy'] >= 0.6]

7     1.000000
10    0.600000
13    0.714286
14    0.833333
15    0.777778
22    0.666667
Name: c: 1.5; k: 0.4,0.5,1.5; xy, dtype: float64

In [116]:
# create list of sequences where performance worse than baseline
rows_sequences = []
rows_sequences.append(results_new['sequence'].iloc[7])
rows_sequences.append(results_new['sequence'].iloc[10])
rows_sequences.append(results_new['sequence'].iloc[13])
rows_sequences.append(results_new['sequence'].iloc[14])
rows_sequences.append(results_new['sequence'].iloc[15])
rows_sequences.append(results_new['sequence'].iloc[22])
rows_sequences

['hscp', 'pnsgb', 'nfpymbs', 'mcbatp', 'pcsbaoweg', 'bca']

In [176]:
results_new['c: 1.5; k: 0.4,0.5,1.5; xy']

0       0.200000
1       0.200000
2       0.000000
3       0.200000
4       0.000000
5       0.400000
6       0.500000
7       1.000000
8       0.000000
9       0.333333
10      0.600000
11      0.500000
12      0.250000
13      0.714286
14      0.833333
15      0.777778
16      0.400000
17      0.000000
18      0.400000
19      0.400000
20      0.000000
21      0.250000
22      0.666667
23      0.250000
mean    0.369808
Name: c: 1.5; k: 0.4,0.5,1.5; xy, dtype: float64

In [183]:
plt.figure(figsize=(6,6))
plt.scatter(results_new['c: 1.5; k: 0.4,0.5,1.5; xy'][:-1], results_mean['sequence'][:-1])
plt.title('all sequences (avg.)')
plt.axvline(x=0.603, color='red')